In [6]:
from glob import glob
import subprocess as sp
import sep
import fitsio
import numpy as np
import math
import time

In [8]:
files = glob('data/*.fits')
start = time.time()
for u in range(100):
    begin = time.time()
    fits = fitsio.FITS(files[u])
    metadata = fits[1].read()
    Magnitude_zeropoint = metadata['mag'][0,0] + 2.5*math.log10(metadata['flux'][0,0])
    mag = metadata['mag'][0,0]

    data = fitsio.read(files[u])
    m, s = np.mean(data[0]), np.std(data[0])
    bkg = sep.Background(data[0],bw=42, bh=42, fw=3, fh=3, fthresh=0.0)
    bkg_image = bkg.back()
    bkg_rms = bkg.rms()
    data_sub = data[0] - bkg
    objects = sep.extract(data_sub, 1.2 , err=bkg.globalrms)
    flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                         3.0, err=bkg.globalrms, gain=1.0)
    center = []
    for v in range(len(objects['x'])):
        if objects['x'][v] < 68 and objects['x'][v] > 60 and objects['y'][v] > 60 and objects['y'][v] < 68:
            center.append(objects['x'][v])   
        
    

    Object_1 = '''
 0) sky                    #  object type
 1) 1.3920      1          #  sky background at center of fitting region [ADUs]
 2) 0.0000      0          #  dsky/dx (sky gradient in x)
 3) 0.0000      0          #  dsky/dy (sky gradient in y)
 Z) 0                      #  output option (0 = resid., 1 = Don't subtract) 
'''
    
                   
                   
    if len(center)== 0 :
        Object_2 = '''
 0) sersic                 #  object type
 1) 64  64  1 1  #  position x, y
 3) 20     1          #  Integrated magnitude	
 4) 5.1160      1          #  R_e (half-light radius)   [pix]
 5) 4.2490      1          #  Sersic index n (de Vaucouleurs n=4) 
 6) 0.0000      0          #     ----- 
 7) 0.0000      0          #     ----- 
 8) 0.0000      0          #     ----- 
 9) 0.7570      1          #  axis ratio (b/a)  
10) -60.3690    1          #  position angle (PA) [deg: Up=0, Left=90]
 Z) 0                      #  output option (0 = resid., 1 = Don't subtract) 
'''
    else:
        Object_2 = ""
        
                   
       
                   

    Object_add = ''
    for i in range(len(objects['x'])):
        Object_add += '''
 0) sersic                 #  object type
 1) ''' + str(int(objects['x'][i])) + ' ' + str(int(objects['y'][i])) + '''  1 1  #  position x, y
 3) ''' + str(Magnitude_zeropoint - 2.5*math.log10(objects['flux'][i])) + '''     1          #  Integrated magnitude	
 4) 5.1160      1          #  R_e (half-light radius)   [pix]
 5) 4.2490      1          #  Sersic index n (de Vaucouleurs n=4) 
 6) 0.0000      0          #     ----- 
 7) 0.0000      0          #     ----- 
 8) 0.0000      0          #     ----- 
 9) 0.7570      1          #  axis ratio (b/a)  
 10) -60.3690    1          #  position angle (PA) [deg: Up=0, Left=90]
 Z) 0                      #  output option (0 = resid., 1 = Don't subtract) 
'''
        




    N_In = str(files[u])
    N_out = str(files[u].replace('/balrog_cutouts_','imgblock_')) 

    front = '''===============================================================================
# IMAGE and GALFIT CONTROL PARAMETERS
A) ''' + N_In + '''      # Input data image (FITS file)
B) ''' + N_out + '''       # Output data image block
C) none                # Sigma image name (made from data if blank or "none") 
D) psf.fits   #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) none                # Bad pixel mask (FITS image or ASCII coord list)
G) none                # File with parameter constraints (ASCII file) 
H) 1    128   1    128   # Image region to fit (xmin xmax ymin ymax)
I) 100    100          # Size of the convolution box (x y)
J) ''' + str(Magnitude_zeropoint) + '''              # Magnitude photometric zeropoint 
K) 0.038  0.038        # Plate scale (dx dy)    [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 0                   # Choose: 0=optimize, 1=model, 2=imgblock, 3=subcomps

''' 
    end = '''
================================================================================'''
    All =  front + Object_1 + Object_2 + Object_add + end
    
    
    outfile = open("doge.txt", "w")
    outfile.write(All)
    outfile.close()
    
    sp.check_call("./galfit doge.txt", shell = True)
    ending = time.time()
    print('file' + str(u)+ ':  '+ str(ending - begin))

    
end = time.time()
print('total: '+ str(end - start))

file0:  0.0052716732025146484
file1:  0.005059003829956055
file2:  0.0039441585540771484
file3:  0.003808259963989258
file4:  0.004100799560546875
file5:  0.003523111343383789
file6:  0.003692150115966797
file7:  0.0037560462951660156
file8:  0.0047931671142578125
file9:  0.0034379959106445312
file10:  0.004149913787841797
file11:  0.003754854202270508
file12:  0.0040018558502197266
file13:  0.0041768550872802734
file14:  0.0035529136657714844
file15:  0.003392934799194336
file16:  0.0037267208099365234
file17:  0.0037009716033935547
file18:  0.006153106689453125
file19:  0.005155801773071289
file20:  0.0033729076385498047
file21:  0.0037970542907714844
file22:  0.0033180713653564453
file23:  0.0036249160766601562
file24:  0.0037407875061035156
file25:  0.0032892227172851562
file26:  0.003325939178466797
file27:  0.0039370059967041016
file28:  0.0041980743408203125
file29:  0.0032410621643066406
file30:  0.0033330917358398438
file31:  0.003622770309448242
file32:  0.0051920413970947266